https://www.stats.ox.ac.uk/~snijders/siena/Lazega_lawyers_data.htm   data set taken from

https://www.stats.ox.ac.uk/~snijders/siena/siena_datasets.htm    main page of SIENA

https://scholar.google.com/citations?user=xqefLxQAAAAJ&hl=en&oi=sra  <<< Authors which provide the data set 

In [ ]:
from IPython.display import HTML

In [ ]:
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# The raw code for this IPython notebook is by default hidden for easier reading.
# To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
import pprint
import pickle
import warnings
import itertools
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import networkx  as nx
import evaluation as ev
from sklearn import metrics
from numpy import linalg as LA
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from pandas import DataFrame as df
import matplotlib.patches as patches
from sklearn.preprocessing import OneHotEncoder
from collections import OrderedDict, defaultdict
import SEFNAC as sefnac 
from numpy.matlib import rand,zeros,ones,empty,eye
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
pivots = list(range(3, 71, 7))
pivots

[3, 10, 17, 24, 31, 38, 45, 52, 59, 66]

In [ ]:
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, linewidth=150, precision=2)

In [ ]:
def preprocess_Y(Yin):
    
    """
    input:
    - Y: numpy array for Entity-to-feature
    - ncf: Dict. the dict-key is the index of categorical variable V_l in Y, and dict-value is the number of 
    sub-categorie b_v (|V_l|) in categorical feature V_l.
    
    Apply Z-scoring, preprocessing by range, and Prof. Mirkin's 3-stage pre-processing methods.
    return: Original entity-to-feature data matrix, Z-scored preprocessed matrix, 2-stages preprocessed matrix,
    3-stages preprocessed matrix and their coresponding relative contribution
    """ 
    
    TY = np.sum(np.multiply(Yin, Yin))  # data scatter, the letter T stands for data scatter
    TY_v = np.sum(np.multiply(Yin, Yin), axis=0)  # feature scatter 
    Y_rel_cntr = TY_v/TY  # relative contribution
    
    Y_mean = np.mean(Yin, axis=0)
    Y_std = np.std(Yin, axis=0)
    
    Yz = (Yin - Y_mean) / Y_std # Z-score
    TYz = np.sum(np.multiply(Yz, Yz))
    TYz_v = np.sum(np.multiply(Yz, Yz), axis=0)
    Yz_rel_cntr = TYz_v / TYz

    scale_min_Y = np.min(Yin, axis=0); scale_max_Y = np.max(Yin, axis=0); rng_Y = scale_max_Y - scale_min_Y
    
    Yrng = (Yin - Y_mean)/ rng_Y  # 3 steps preprocessing (Range-without follow-up division)
    TYrng = np.sum(np.multiply(Yrng, Yrng))
    TYrng_v = np.sum(np.multiply(Yrng, Yrng), axis=0)
    Yrng_rel_cntr = TYrng_v/TYrng
    
    # This section is not used for synthetic data, because no categorical data is generated.
    Yrng_rs = deepcopy(Yrng) # 3 steps preprocessing (Range-with follow-up division)
    for k, v in ncf.items(): 
        Yrng_rs[:, int(k)] = Yrng_rs[:, int(k)]/np.sqrt(int(v))  # : int(k)+ int(v)
        
#     Yrng_rs = (Y_rescale - Y_mean)/ rng_Y
    TYrng_rs = np.sum(np.multiply(Yrng_rs, Yrng_rs))
    TYrng_v_rs = np.sum(np.multiply(Yrng_rs, Yrng_rs), axis=0)
    Yrng_rel_cntr_rs = TYrng_v_rs/TYrng_rs
    
    return Y_rel_cntr, Yz, Yz_rel_cntr, Yrng, Yrng_rel_cntr, Yrng_rs, Yrng_rel_cntr_rs

In [ ]:
def preprocess_P(Pin):

    """
    input: Adjacency matrix
    Apply Uniform, Modularity, Lapin preprocessing methods.
    return: Original Adjanceny matrix, Uniform preprocessed matrix, Modularity preprocessed matrix, and 
    Lapin preprocessed matrix and their coresponding relative contribution
    """ 
    N, V = Pin.shape
    P_sum_sim = np.sum(Pin)
    P_ave_sim = (np.sum(Pin) / (N*(V-1)))
    cnt_rnd_interact = np.mean(Pin, axis=1) # constant random interaction 
    
    
    # Uniform method
    Pu = Pin - cnt_rnd_interact
    Pu_sum_sim = np.sum(Pu)
    Pu_ave_sim = (np.sum(Pu) / (N*(V-1)))
    
    # Modularity method (random interaction)
    P_row = np.sum(Pin, axis=0) 
    P_col = np.sum(Pin, axis=1)
    P_tot = np.sum(Pin)
    rnd_interact = np.multiply(P_row, P_col)/P_tot # random interaction formula
    Pm = Pin - rnd_interact
    Pm_sum_sim = np.sum(Pm)
    Pm_ave_sim = (np.sum(Pm) /(N*(V-1)))
    
    # Lapin (Laplacian Inverse Transform)
    # Laplacian
    
    r, c = Pin.shape
    P_ = (Pin + Pin.T)/2
    Pr = np.sum(P_, axis=1)
    D = np.diag(Pr)
    D = np.sqrt(D)
    Di = LA.pinv(D)
    L = eye(r) - Di@Pin@Di
    
    PL_sum_sim = np.sum(L)
    PL_ave_sim = (np.sum(L)/(N*(V-1)))
    
    # pseudo-inverse transformation
    L = (L + L.T)/2
    M, Z = LA.eig(L)
    ee = np.diag(M)
    ind = list(np.nonzero(ee>0)[0])
    Zn = Z[ind, ind]
    Mn = np.asarray(M[ind])
    Mi = np.asarray(1/Mn)
    lapin =  Zn@Mi@Zn
#     print("Zn:", Zn.shape)
#     print("Mi", Mi.shape)
#     print("Zn.T:", Zn.T.shape)
#     lapin =  Zn@Mi@Zn
    
    return P_sum_sim, P_ave_sim, Pu, Pu_sum_sim, Pu_ave_sim, Pm, Pm_sum_sim, Pm_ave_sim, L, PL_sum_sim, PL_ave_sim

In [ ]:
def draw_adjacency_matrix(G, title="", node_order=None, partitions=[], colors=[], partitions_name=[]):

    """
    - G is a netorwkx graph
    - node_order (optional) is a list of nodes, where each node in G
          appears exactly once
    - partitions is a list of node lists, where each node in G appears
          in exactly one node list
    - colors is a list of strings indicating what color each
          partition should be
    If partitions is specified, the same number of colors needs to be
    specified.
    """
    
    adjacency_matrix = nx.to_numpy_matrix(G, dtype=np.bool, nodelist=node_order)

    #Plot adjacency matrix 
    fig = plt.figure(figsize=(5, 5)) # in inches
    plt.title(title, fontsize=15)
    plt.imshow(adjacency_matrix,
               cmap="cool_r",  # spring, cool
               interpolation="None",
               origin='lower',)  # extent=[0, 70, 0, 70] to chane the x and y axises value

    # The rest is just if you have sorted nodes by a partition and want to
    # highlight the module boundaries
    assert len(partitions) == len(colors)
    ax = plt.gca()
    
    for partition, color in zip(partitions, colors):
        rec_cord = 0
        txt_cord = 10
        p = 0    
        for module in partition:
            ax.add_patch(patches.Rectangle((rec_cord, rec_cord),
                                          len(module), # Width
                                          len(module), # Height
                                          facecolor="none",
                                          edgecolor=color,
                                          linewidth="1",
                                          )),
            if len(v) != 0:
                ax.text(txt_cord, txt_cord, partitions_name[p], ha="center", va="center", color="k", fontsize=16,)
            rec_cord += len(module)
            txt_cord += len(module)
            p += 1

In [ ]:
def draw_several_adjacencies(Graph, dict_of_features, rows, cols, ind, size_of_figures=(20, 10)):
    
    """
    This is a very dirty extention of draw_adjacency_matrix funtion to be able
    to see all of the matrices all together into one figure using plt.subplots().
    """
    fi = 0
    fig, ax = plt.subplots(figsize=size_of_figures) 

    for k, v in dict_of_features.items():
        gnfl = group_feature_to_lists(ELattr[:, fi])  # Grouping Nodes according to the Feature to Lists
        nodes_gnfl_ordered = [node for group in gnfl for node in group]  # Create a list of all nodes sorted by coresponding feature

        adjacency_matrix = nx.to_numpy_matrix(Graph, dtype=np.bool, nodelist=nodes_gnfl_ordered)

        plt.subplot(rows, cols, ind)
        plt.title(k, fontsize=50)

        #Plot adjacency matrix 
        plt.imshow(adjacency_matrix,
                   cmap="cool_r",  # spring, cool
                   interpolation="None",
                   origin='lower',)  # extent=[0, 70, 0, 70] to chane the x and y axises value

        # The rest is just if you have sorted nodes by a partition and want to
        # highlight the module boundaries
        colors=["black"]

        assert len([gnfl]) == len(colors)
        ax = plt.gca()
        p = 0
        for partition, color in zip([gnfl], colors):
            rec_cord = 0
            txt_cord = 10
            for module in partition:
                ax.add_patch(patches.Rectangle((rec_cord, rec_cord),
                                              len(module), # Width
                                              len(module), # Height
                                              facecolor="none",
                                              edgecolor=color,
                                              linewidth="3",))
                if len(v) != 0:
                    ax.text(txt_cord, txt_cord, v[p], ha="center", va="center", color="k", fontsize=38,)
                rec_cord += len(module)
                txt_cord += len(module)
                p += 1
        ind += 1
        fi += 1
    return None

In [ ]:
def group_feature_to_lists(input_feature):
    by_feature_value = defaultdict(list)
    for node_index, attribute_value in enumerate(input_feature):
        by_feature_value[attribute_value].append(node_index)
    return by_feature_value.values()

In [ ]:
def apply_SEFNAC(Yi, Pi, ro_g, ro_f, data_name):
    """
    input: entity-to-feature matrix Yi, Adjacency matrix Pi, ro_f, ro_g trade-off factor
    between data matrices, data_name.
    applies simultanouse anamalouse (network) clustering algorithm, sa(n)c.
    return: clustering_results as it is appeared and sorted clustering results based on the number
    of members in each cluster in ascending order.
    """
    out_ms, out_ms_sorted, contingency_table = OrderedDict([]), OrderedDict([]), OrderedDict([])
    N, V = Yi.shape
    
    path = data_name + "-" + str(ro_g) + "-" + str(ro_f)
    print("new path:", path,)

    for pivot in pivots:
        path_ = path + "-pivot:" + str(pivot)
        print("pivot:", pivot)
        
        out_ms[path_] = sefnac.run_ANomalous_Cluster(pivot=pivot, Y=Yi, P=Pi, rho_f=ro_f, rho_g=ro_g)
        length_labels_pred, sorted_out_ms = sefnac.sorting_results(out_ms[path_])
        out_ms_sorted[path_] = sorted_out_ms.values()

    result_path = data_name + "-" + "-ro-g=" + str(ro_g) + "-ro_f=" + str(ro_f)
    return out_ms, out_ms_sorted

In [ ]:
def QScaD(Y_, P_, ro_f, ro_g, results):
    
    """
    input: the original entity-to-feature matrix and the one which  is used for clustering process.
    The original adjacency matrix and the one which is used for the clustering process.
    And finally, the sorted clustering results dictionary.
    computes and prints out grand means of data matrices, "features mean" in each cluster, 
    cluster contribution, relative_difference
    return: Dict of Dict 
    """
    
    QScaD = {}
    Y = np.multiply(Y_, ro_f)
    P = np.multiply(P_, ro_g)
    for k, v in results.items():
        QScaD[k] = {}
        TY = np.sum(np.multiply(Y, Y))
#         Tv = np.mean(np.multiply(Y, Y), axis=0)
        Tv = np.sum(np.multiply(Y, Y), axis=0)
        PSs = np.sum(np.multiply(P, P))
        Cent, Ck, Bcnt, Bk, Rcnt, Rk, Dcnt, Dk, RCI, Rck, Bv, YExpl, YUnExpl, PExpl, \
        PUnExpl, = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}
        
        i = 1
        for indices in v:
            
            Cent[str(i)] = np.mean(Y[indices, :], axis=0)
            Ck[str(i)] = np.mean(Y[indices, :], )
            B_kv = np.divide(len(indices) * np.mean(np.power(Y[indices, :], 2), axis=0), 1)  # B_kv
            Bcnt[str(i)] = B_kv
            Bk[str(i)] = np.sum(B_kv)  # B_k+
            
            B_kRv = np.divide(B_kv, Tv)  # B(k|v) 
            Rcnt[str(i)] = B_kRv
            Rk[str(i)] = np.sum(B_kRv)
            
            G_kRv = np.subtract(B_kRv, np.divide(np.sum(B_kRv), TY))  # g(K|v)
            Dcnt[str(i)] = G_kRv
            Dk[str(i)] = np.sum(G_kRv)
            
            Q_kRv = np.subtract(np.divide(np.multiply(TY, B_kv), np.multiply(Tv, np.sum(B_kv))), 1)  # q(k|v)
            RCI[str(i)] = Q_kRv
            Rck[str(i)] = np.sum(Q_kRv)
            
            CCo = np.reshape([P[i, j] for i in indices for j in indices], (len(indices), -1)) # Cluster Co-occure
            CSs = np.sum(CCo)  # Cluster Similarity Sum 
            Lambda = np.divide(CSs, len(indices)**2)
            
            PExpl[str(i)] = Lambda * CSs
            PUnExpl[str(i)] = PSs - (Lambda * CSs)
            i += 1
        
        QScaD[k]['Cent'] = Cent 
        QScaD[k]['Ck'] = Ck
        QScaD[k]['Bcnt'] = Bcnt
        QScaD[k]['Rcnt'] = Rcnt
        QScaD[k]['Dcnt'] = Dcnt
        QScaD[k]['RCI'] = RCI
        
        QScaD[k]['Bk'] = Bk
        QScaD[k]['Rk'] = Rk
        QScaD[k]['Dk'] = Dk
        QScaD[k]['Rck'] = Rck

        QScaD[k]['PExpl'] = PExpl
        QScaD[k]['PUnExpl'] = PUnExpl
        
        
    return QScaD

## Testing Strategy 

We investigate the impact of different preprocessoing methods on feature data and on graph's data. These methods are :

* For feature data: Z-scoring, renage scale, and 3-stage with follow up rescale.
* For Graph data:Uniform, Modularity and Lapin


3-stage preprocessing technique normalized the data contribution and save the shape of data scatter.

We also investigate the impact of $\rho_{f}$ and $\rho_{g}$. 

To evaluate the clustering results we use Four different methods.

1) Contribution and relative contribution tables

2) Adjusted Rand index ARI

3) Contingency table 

At first step, we determine which set of features contribute each cluster, we also show the explain and unexplain part of graph data in each cluster. However, since the value of the explain and unexplain parts of graph data are not intuitivem to show the effect of graph data we recolor the network regarding the each cluster. 

We also use ARI, and the contingency as another set of criteria. We create the ground truth based on status and office location of laywers. 

### Loading the data

In [3]:
ELfriend = np.loadtxt("../../data/Lawyers/ELfriend.dat")
ELattr = np.loadtxt("../../data/Lawyers/ELattr.dat")
Yin = ELattr[:, 1:].copy()
Pin = ELfriend.copy()

N, V = Yin.shape

print("ELfriend:", type(ELfriend), ELfriend.shape)
print("ELattr:", type(ELattr), ELattr.shape, Yin.shape)

ELfriend: <class 'numpy.ndarray'> (71, 71)
ELattr: <class 'numpy.ndarray'> (71, 8) (71, 7)


### Original Adjacency Matrix

In [ ]:
features = ["sen, sta, gen, off, yif, age, pra, law"]  

features_dict = {"Seniority": "", "Status": ["Partner", "Associate"], "Gender": ["Male", "Female"], 
                 "Office": ["Boston", "Hartford", "Providence"], "Years in Firm": "", "Age": "", 
                 "practice": ["Litigation", "Corporate"], "Law School": ["Harvard, Yale", "ucon", "Other"]} 

print("Features are:")
print("Seniority, status (1=partner; 2=associate), gender (1=man; 2=woman),") 
print("office (1=Boston; 2=Hartford; 3=Providence), years with the firm, age,") 
print("practice (1=litigation; 2=corporate), law school (1: harvard, yale; 2: ucon; 3: other)")
GP = nx.from_numpy_matrix(ELfriend)
draw_adjacency_matrix(G=GP, title="Original Adjacency Matrix")

In [ ]:
labels_nx_org = {}
for i in range(N):
    labels_nx_org[i] = i
    

plt.figure(figsize=(10, 5.5))
pos_org = nx.kamada_kawai_layout(GP)

nx.draw_networkx(G=GP, pos=pos_org, nodelist=list(range(0,N)),
                 labels=labels_nx_org, alpha=0.9, width=.3, node_color='lime')
plt.show()

As one can easily perceive no specific pattern can be detected in the network structure.  In the next step, we sort the adjacency matrix regarding each feature to review the relation between each feature and the graph's structure.

In [ ]:
bins = plt.hist(Yin[:, 3])
plt.title("Histogram of Years with the Firm")
plt.show()
bins

10 years with the firm is chosen as a threshold for categorizing this feature

In [ ]:
bins_ = plt.hist(Yin[:, 4])
plt.title("Histogram of Lawyers Age")
plt.show()
bins

Age of 40, between 40 and 50 years old and greater than 50 years old Lawyers, are the three categories for age.

In [ ]:
features = ["Seniority", "Status", "Gender", "Office", "Years in Firm", "Age", "practice", "Law School"]

Yc = np.zeros([N, 7])

for i in range(N):
    for v in range(V):
        if v == 3 and Yin[i, v] < 10:  
            Yc[i, v] = 1
        elif v == 3 and  10 <= Yin[i, v] < 20:
            Yc[i, v] = 2
        elif v == 3 and  20 <= Yin[i, v]:
            Yc[i, v] = 3
        elif v == 4 and Yin[i, v] < 40:
            Yc[i, v] = 1
        elif v == 4 and  40 <= Yin[i, v] < 50:
            Yc[i, v] = 2
        elif v == 4 and  50 <= Yin[i, v]:
            Yc[i, v] = 3
        else:
            Yc[i, v] = Yin[i, v]

Yc_unique = np.zeros([N, V])
interval = 0
for v in range(V):
    for i in range(N):
        if v == 0:
            Yc_unique[i, v] = Yc[i, v]
        else:
            Yc_unique[i, v] = Yc[i, v] + interval  # even without interval it's ok
    if v != 0:        
        interval += max(list(set(Yc[:, v])))

print("Mod Y Dim:", Yc_unique.shape)

In [ ]:
enc = OneHotEncoder(categories='auto', sparse=False)
Y_oneHot_final = enc.fit_transform(Yc)  # oneHot encoding
print(Y_oneHot_final.shape)

### Mix of categorical and quatitative 

In [ ]:
# Y_oneHot_1 = enc.fit_transform(Yin[:, :3])
# Y_oneHot_2 = enc.fit_transform(Yin[:, 5:])
# Y_oneHot_mix = np.concatenate([Y_oneHot_1, Yin[:, 3:5], Y_oneHot_2, ], axis=1)

### Applying SEFNAC on Yrang and Pm 

In [ ]:
# ncf = {'1':2, '2':2, '3':3, '6':2, '7':3}  # ORIGINAL Y
ncf = {'0':2, '1':2, 
       '2':2, '3':2, 
       '4':3, '5':3, '6':3,
       '9':2, '10':2,
       '11':3, '12':3, '13':3
       }
Y_rel_cntr, Yz, Yz_rel_cntr, Yrng, Yrng_rel_cntr, Yrng_rs, Yrng_rel_cntr_rs = preprocess_Y(Yin=Y_oneHot_final)  # Y_oneHot_mix >> for Mixed!
print("Features: seniority, status (1=partner; 2=associate), gender (1=man; 2=woman), office (1=Boston; 2=Hartford; 3=Providence), years with the firm, age, practice (1=litigation; 2=corporate), law school (1: harvard, yale; 2: ucon; 3: other)")
print(" ")
print("Y relative cntr              :", Y_rel_cntr, sum(Y_rel_cntr))
print("Z-score relative cntr        :", Yz_rel_cntr, sum(Yz_rel_cntr))
print("Yrng relative cntr           :", Yrng_rel_cntr, '%.2f' % sum(Yrng_rel_cntr))
print("rescaled Yrng relative cntr  :", Yrng_rel_cntr_rs, '%2.f' % sum(Yrng_rel_cntr_rs))

In [ ]:
# # ncf = {'1':2, '2':2, '3':3, '6':2, '7':3}  # ORIGINAL Y

# ncf_all = {'0':2, '1':2,
#            '2':2, '3':2, 
#            '4':3, '5':3, '6':3,
#            '7':3, '8':3,
#            '9':2, '10':2,
#            '11':3, '12':3, '13':3
#        }

# Y_rel_cntr_all, Yz_all, Yz_rel_cntr_all, Yrng_all, Yrng_rel_cntr_all, Yrng_rs_all, Yrng_rel_cntr_rs_all = preprocess_Y(Yin=Y_oneHot)
# print("Features: seniority, status (1=partner; 2=associate), gender (1=man; 2=woman), office (1=Boston; 2=Hartford; 3=Providence), years with the firm, age, practice (1=litigation; 2=corporate), law school (1: harvard, yale; 2: ucon; 3: other)")
# print(" ")
# print("Y relative cntr              :", Y_rel_cnt_all, sum(Y_rel_cntr_all))
# print("Z-score relative cntr        :", Yz_rel_cntr_all, sum(Yz_rel_cntr_all))
# print("Yrng relative cntr           :", Yrng_rel_cntr_all, '%.1f' % sum(Yrng_rel_cntr_all))
# print("rescaled Yrng relative cntr  :", Yrng_rel_cntr_rs_all, '%.1f' % sum(Yrng_rel_cntr_rs_all))

### Preprocessing the Adjacency matrix

In [ ]:
P_sum_sim, P_ave_sim, Pu, Pu_sum_sim, Pu_ave_sim, Pm, Pm_sum_sim, Pm_ave_sim, Pl, Pl_sum_sim, Pl_ave_sim = preprocess_P(ELfriend)  # PL, PL_rel_cntr 

# print("sum similarity of P:", P_sum_sim)
print("Ave similarity of P:", '%.2f' % P_ave_sim)

print("sum similarity of Pu:", '%.2f' % Pu_sum_sim)
print("Ave similarity of Pu:", '%.2f' % Pu_ave_sim)

print("sum similarity of Pm:", '%.2f' % Pm_sum_sim)
print("Ave similarity of Pm:", '%.2f' % Pm_ave_sim)

print("sum similarity of Pl:", '%.2f' % Pl_sum_sim)
print("Ave similarity of Pl:", '%.2f' % Pl_ave_sim)

### The impact of applying different preprocessing methods

As one can easily perceive applying different preprocessing method does not affect the clustering results considerably. Taking a look at the values of ARI for different cases as well as comparing the contingency tables can prove this claim. However, they can slightly improve manifesting the role of each feature contribution in clustering preprocess. And this is why in the rest of this report we use the case which the features' data is preprocessed by utilizing the 3-stage method and the graph's data is preprocessed utilizing Modularity method. 


In [ ]:
GPm = nx.from_numpy_matrix(Pm)
GPu = nx.from_numpy_matrix(Pu)
GLa = nx.from_numpy_matrix(Pl)
# draw_adjacency_matrix(GP, title="Without normalization")
# draw_adjacency_matrix(GPm, title="Normalized with Modulirty")
# draw_adjacency_matrix(GPu, title="Normalizad with uniform method")
# draw_adjacency_matrix(GLa, title="Normalizad with Lapin")

#### Sorting the original Adjanceny Matrix regarding different features

In [ ]:
draw_several_adjacencies(Graph=GP, dict_of_features=features_dict, rows=2, cols=4, ind=1, size_of_figures=(50, 25))

By sorting the adjancency matrix redarding each feature and reviewing the paper which introduces the data set, we create the ground truth using status feature and office location.

### Create a ground Truth based on Status and Office location (features)
#### Sorted Case

In [ ]:
unique_labels = []
for i in range(N):
    tmp = ELattr[i, [1, 3]].tolist()
#     print(tmp, type) 
    if tmp not in unique_labels:
        unique_labels.append(tmp)
unique_labels

In [4]:
label = 1
status = set(ELattr[:, 1])  # sorted(set(ELattr[:, 1]))
office = set(ELattr[:, 3]) 
r, c = ELattr.shape
all_possible_labels = list(itertools.product(status, office))

r, c = ELattr.shape
labels_true = []
labels_true_final = np.zeros(N)
labels_true_sian = []


for lb in all_possible_labels:
    tmp_labels, tmp_sian = [], []
    at_least_one_label = False
    for i in range(N):
        if ELattr[i, 1] == lb[0] and ELattr[i, 3] == lb[1]:
            at_least_one_label = True
            tmp_labels.append(label)
            labels_true_final[i] = label
    if at_least_one_label is True:
        label += 1
    if len(tmp_labels) != 0:
        labels_true.append(tmp_labels)
        labels_true_sian.append(tmp_sian)
        
labels_true_final = labels_true_final.tolist()
len(labels_true),

(6,)

In [ ]:
# np.save("labels_true_final_LAWYERS.npy", labels_true_final)

### Apply SEFNAC algorithm on not preprocessed data

In [ ]:
ro_g = 1 # trade-off factor for interaction between entity-to-feature matrix and proximity
ro_f = 1 # trade-off factor for interaction between entity-to-feature matrix and proximity

ro_g_ = 2 # trade-off factor for interaction between entity-to-feature matrix and proximity
ro_f_ = 0.2 # trade-off factor for interaction between entity-to-feature matrix and proximity

ro_g__ = 0.2 # trade-off factor for interaction between entity-to-feature matrix and proximity
ro_f__ = 2 # trade-off factor for interaction between entity-to-feature matrix and proximity

Y_to_use = Yrng # Y_oneHot1
P_to_use = Pu

MS, _ = apply_SEFNAC(Yi=Y_to_use, Pi=P_to_use, ro_g=ro_g, ro_f=ro_f, data_name="ELfriend")
# MS_, MSS_ = apply_SEFNAC(Yi=Y_oneHot_final, Pi=Pin, ro_g=ro_g_, ro_f=ro_f_, data_name="ELfriend")
# MS__, MSS__ = apply_SEFNAC(Yi=Y_oneHot_final, Pi=Pin, ro_g=ro_g_, ro_f=ro_f_, data_name="ELfriend")


# T = QScaD(Y_=Yrng_rs, P_=Pm, ro_f=ro_f, ro_g=ro_g, results=MSS)
# T_ = QScaD(Y_=Yrng_rs, P_=Pm, ro_f=ro_f_, ro_g=ro_g_, results=MSS_)
# T__ = QScaD(Y_=Yrng_rs, P_=Pm, ro_f=ro_f__, ro_g=ro_g__, results=MSS__)


##### Adjusted Rand index ARI:

Given the knowledge of the ground truth class assignments, as it is described previously, and the clustering algorithm assignments, ARI is a function that measures the similarity of the two assignments, ignoring permutations. Random (uniform) label assignments have a ARI score close to 0.0 for any value of n_clusters and n_samples. Bounded range [-1, 1]: negative values are bad (independent labelings), similar clusterings have a positive ARI, 1.0 is the perfect match score.


In [ ]:
ari_val, nmi_val, num_clust = [], [], []
for k, v in MS.items():
    if int(k.split(":")[-1]) in pivots:
        labels_pred = np.zeros([N])
        ind = 1
        for kk, vv in v.items():
            for vvv in vv:
                labels_pred[vvv] = ind
                num_clust
            ind += 1
        num_clust += [ind-1]
        ari = metrics.adjusted_rand_score(labels_true=labels_true_final, labels_pred=labels_pred)
        ari_val.append(ari)
        ami = metrics.adjusted_mutual_info_score(labels_true=labels_true_final, labels_pred=labels_pred)
        nmi = metrics.normalized_mutual_info_score(labels_true=labels_true_final, labels_pred=labels_pred)
        nmi_val.append(nmi)
        print("results:", k, "ari:", '%.2f' % ari, "nmi:", '%.2f' % nmi,)  # "ami:", '%.2f' % ami)

In [ ]:
ari_val = np.asarray(ari_val)
nmi_val = np.asanyarray(nmi_val)
num_clust = np.asanyarray(num_clust)

print("   ARI   ", "    NMI  ", "  NUM_CLUST")
print("MEAN  STD", " MEAN STD", "  MEAN STD")
print("%.2f" % np.mean(ari_val), "%.2f" % np.std(ari_val), "%.2f" % np.mean(nmi_val), "%.2f" % np.std(nmi_val), "%.2f" % np.mean(num_clust), "%.2f" % np.std(num_clust))

### all the values are one hot encoded

### Equal weight for graph and feature data

###### To Investigate the clustering results regarding the features run the below cell 

In [ ]:
# for i in pivots:
#     if i == np.argmax(ari_val):
#         print("features:       ", features[1:])
#         nc = 1
#         print("features in:    ", features[0].split(", ")[1:])
#         for k, v in T['ELfriend-1-1-pivot:' + str(i)]['Cent'].items():
#             print("pivot:", i)
#             print('cluster:', nc, 'Cent:', T['ELfriend-1-1-pivot:' + str(i)]['Cent'][str(nc)], '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['Ck'][str(nc)],)
#             print('cluster:', nc, 'Bcnt:', T['ELfriend-1-1-pivot:' + str(i)]['Bcnt'][str(nc)], '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['Bk'][str(nc)],)
# #             print('cluster:', nc, 'Rcnt:', T['ELfriend-1-1-pivot:' + str(i)]['Rcnt'][str(nc)], '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['Rk'][str(nc)],)
# #             print('cluster:', nc, 'Dcnt:', T['ELfriend-1-1-pivot:' + str(i)]['Dcnt'][str(nc)], '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['Dk'][str(nc)])
#             print('cluster:', nc, 'RCI: ', T['ELfriend-1-1-pivot:' + str(i)]['RCI' ][str(nc)], '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['Rck'][str(nc)])
#             print('cluster:', nc, 'P-Expl:' ,'%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['PExpl'][str(nc)], 'P-UnExpl:', '%.2f' % T['ELfriend-1-1-pivot:' + str(i)]['PUnExpl'][str(nc)])
#             print(" ")
#             nc += 1
#         print('***************************************************************************')
#         print(" ")


In [ ]:
# colors = ['fuchsia','green', 'blue', 'aqua', 'lime', 'silver', 'red',
#           'grey', 'maroon', 'purple', 'yellow', 'green', 'lime', 'olive', 
#           'yellow', 'navy', 'aqua', 'black', 'silver', 'grey', 'maroon', 'olive', 'teal',]
# shapes = ['s', 'o', '^', '>', 'v', '<', 'd', 'p', 'h', '8']

# for k, v in MS.items():
#     if int(k.split(":")[-1]) == np.argmax(ari_val):
#         print("number of clusters:", len(v))
#         s = 1
#         nodes_size = []
#         colors_map = []
#         nodes_list = []
#         shapes_map = []
#         for kk, vv in v.items():
#             nodes_list += vv
#             for vvv in vv:
#                 nodes_size += [s*100]
#                 colors_map += [colors[s-1]] 
#                 shapes_map += shapes[s-1]
#             s += 1
#         plt.figure(figsize=(13, 6.5))
# #         pdot = nx.drawing.nx_pydot.to_pydot(GP)
# #         for i, node in enumerate(pdot.get_nodes()):
# #             node.set_label("n%d" % i)
# #             node.set_shape(shapes_map[i-1])
# #             node.set_fillcolor(colors_map[i-1])
# #             node.set_color(colors_map[i])

# #         for i, edge in enumerate(pdot.get_edges()):
# #             edge.set_color(colors_map[1])

        
#         nx.draw_networkx(G=GP, pos=pos_org, nodelist=nodes_list,  # list(range(0,N))
#                          labels=labels_nx_org, alpha=0.9, width=.1, 
#                          node_color=colors_map, 
#                          node_size=nodes_size)
        
#         plt.title("cluster results:" + k, fontsize=20)
#         plt.show()
# #         png_path = k + ".png"
# #         pdot.write_png(png_path)
#         print("*************************************************************************************************************")



## Evaluating the clustering results for the case of not preprocessed data 

### Contingency Tables For Raw Data

In [ ]:
plt.figure(figsize=(7, 4.5))
plt.plot(ari_val)
plt.xlabel("pivots")
plt.ylabel("ARI")
plt.grid(True)
# plt.legend("ARI")
plt.show()

In [ ]:
for k, v in MS.items():
    if int(k.split(":")[-1]) == np.argmax(ari_val):
        
        labels_pred = []
        ind = 1
        for kk, vv in v.items():
            for vvv in vv:
                labels_pred.append(ind)
            ind += 1

        sk_contingency_matrix = contingency_matrix(labels_true=labels_true_final, 
                                                   labels_pred=labels_pred, eps=0.001)

        row_cont, col_cont = sk_contingency_matrix.shape
        sk_contingency_table = np.zeros([row_cont + 2, col_cont + 1])
        rows, cols = sk_contingency_table.shape

        # adding marginal row and columns to convert the contingency matrix to contingency table                                                  
        sk_contingency_table[1:-1, 0:-1] = sk_contingency_matrix

#         marginal row values (Nt+)                                                                                                               
#         for row in range(rows):
#             sk_contingency_table[row, -1] = int(sum(sk_contingency_table[row, :]))

        # marginal column values (N+u)                                                                                                            
        for col in range(cols):
            
            sk_contingency_table[-1, col] = int(sum(sk_contingency_table[:, col]))
#             if 0 < col < cols-1:
#                 print(col,  len(labels_true[col-1]))
#                 sk_contingency_table[0, col] = len(labels_true[col-1])

        print(sk_contingency_table)
        print(" ")

# Cluster Analysis

#### Note:

* The Description in this section may vary in each run, however, still it could be a good sample of analyzing the clustering results with the help Quetelet Relative index and graph structure

Description:

A closer look at the entity-to-feature data matrix of the first cluster, reveals that all the members of this cluster have the same status, that is, they work in associate status. Gender does not provide any specific information. Moreover, all members of this cluster are graduated from Ucon or other Universities except entity number 37. By taking into account the graph's data (nodes with fuchsia) and looking at the links in this cluster two important facts can be understood. Firstly, all these entities are highly connected with each other. Secondly, entity number 37, has a very strong connection with all the members of this cluster. (Regarding the years he spent in the firm and his connection it seems as if he is a person in charge in among the lawyer of this cluster.) \qed.


The most contributing features in the second cluster are "age" and "years in the firm". The average age of the member of the cluster is 52.4 and the average years in the firm is 21. Besides that, the status all the member of this cluster are partners. Looking at the links among them the members of this cluster (green nodes) one can easily see how dense the connections in this cluster are. (A cluster of owners in Boston and Hartford offices).

Another interesting phenomenon in this cluster is considering node number 13 in this cluster which show the meaningfulness of the clustering result because in spite of joining to the company very recently but she is one the Lawyers with partner status who has sufficient links with other Lawyers in Hartford office. \qed

The most contributing feature in the third cluster is office location. Moreover, Except node number 46, of all members of this cluster, are corporate attorneys. Though, since the members of this cluster, do not have strong connections among themselves and between the lawyers of other clusters, consequently, the link between node number 46 and node number 43, and the connection between 43 and 45 could be the reason of considering this node in the third cluster. \qed  

(think assging more weight to links should seperate this cluster into two clusters because of office feature in this cluter.)

Office, Status, Law School and practice are sorted in descending order regarding their contribution to cluster number four. To be more precise, This cluster is a group of male litigation attorneys who work in a partnership framework. All of them expect node number 35, work in Hartford office. Looking at graphs structure, blue nodes can justify the reason of considering node number 35 in this cluster -- because it has links with node number 30 which is a kind of hub node in this cluster. \qed

The fifth cluster can be considered as a very tight cluster of male corporate lawyers of Boston office with practically the same age and the same years of experience of working in the firm. The nodes of this cluster are demonstrated with Cyan colour. \qed


This cluster consists of a man of the age 34 and a woman of age 53. Their working status is partner, they both work in Boston office and both are litigation attorneys who have stron connectios with different Lawyers of different cluster and with themselves as well. Moreover, they have practically the same years of experience in the firm. Nodes of this cluster are shown with olive colour. \qed

## Results of Applying CESNA on this data set
#### Loading the save results and other post processing methods


In [ ]:
# with open("/home/Soroosh/snap/examples/cesna/lawyers/Lawyers"+ str(0) +"cmtyvv.txt", 'r') as fp:
#     cesna = fp.readlines()

# cesna_indices = []
# for i in cesna:
#     cesna_indices.append([int(ii) for ii in i.split()])

In [8]:
ari_cesna_total, nmi_cesna_total = [], []

for i in range(10):
    with open("/home/Soroosh/snap/examples/cesna/lawyers/Lawyers"+ str(i) +"cmtyvv.txt", 'r') as fp:
        cesna = fp.readlines()
        
    cesna_indices = []
    for i in cesna:
        cesna_indices.append([int(ii) for ii in i.split()])

    cesna_labels = np.zeros([N])
    label = 1
    for lst in cesna_indices:
        for l in lst:
            cesna_labels[l-1] = label
        label += 1

    ari_cesna = metrics.adjusted_rand_score(labels_true=labels_true_final, labels_pred=cesna_labels)
    ami_cesna = metrics.adjusted_mutual_info_score(labels_true=labels_true_final, labels_pred=cesna_labels)
    nmi_cesna = metrics.normalized_mutual_info_score(labels_true=labels_true_final, labels_pred=cesna_labels)

    print("results of applying CESNA:", "ari:", '%.2f' % ari_cesna,  "nmi:", '%0.2f' % nmi_cesna)
    ari_cesna_total.append(ari_cesna)
    nmi_cesna_total.append(nmi_cesna)
    
print("Final:", "%.2f" % np.mean(ari_cesna_total), "%.2f" % np.std(ari_cesna_total), "%.2f" % np.mean(nmi_cesna_total), "%.2f" % np.std(nmi_cesna_total))

results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
results of applying CESNA: ari: 0.28 nmi: 0.49
Final: 0.28 0.00 0.49 0.00


In [ ]:
# ari_cesna_total, nmi_cesna_total = [], []

# for i in range(10):
    
#     with open("/home/Soroosh/snap/examples/cesna/lawyers/Lawyers"+ str(i) +"cmtyvv.txt", 'r') as fp:
#         cesna = fp.readlines()

#     cesna_indices = []
#     for i in cesna:
#         cesna_indices.append([int(ii) for ii in i.split()])

#     cesna_indices.sort(key=len, reverse=True)
#     cesna_indices_sorted = deepcopy(cesna_indices)
#     cesna_results_info = []
#     c = 1
#     for i in cesna_indices_sorted:
#         cesna_results_info.append((c, len(i)))
#         c += 1
#     cesna_results_info


#     cesna_results_unique_indices = []
#     for i in range(len(cesna_indices_sorted)):
#         tmp =cesna_indices_sorted[i]
#         for j in range(i):
#             tmp = set(tmp).difference(cesna_indices_sorted[j])
#         cesna_results_unique_indices.append(list(tmp))



#     diff = []  # add those nodes which are ignored by cesna 
#     cesna_indices_sorted_flat = [item for sublist in cesna_results_unique_indices for item in sublist]
#     for i in range(1, N+1):
#         if not i in cesna_indices_sorted_flat:
#             diff += [i]

# #     print(list(set(cesna_indices_sorted_flat)), len(list(set(cesna_indices_sorted_flat))))
# #     print(" ")
# #     print("diff:", diff, len(diff))


#     cesna_labels = np.zeros([N])
#     label = 0
#     for lst in cesna_results_unique_indices:
#         label += 1
#         for l in lst:
#             cesna_labels[l-1] = label

# #     print("cesna_labels:", len(cesna_labels))


#     ##############################################################################################################
#     # Remove/Add some elemenets from the output of CESNA results to compensate that
#     # 10% of nodes which explained in section "Chososing the number of communities"
#     ##############################################################################################################

#     # diff_true_pred_labels = len(labels_true_final) - len(cesna_labels) 
#     # print("diff_pred_true_labels:", diff_true_pred_labels)

#     # # if diff_true_pred_labels > 0:
#     # #     for d in diff_true_pred_labels:
#     #         cesna_labels[d-1] = 50
#     # # elif diff_true_pred_labels < 0:
#     # #         cesna_labels.pop()

#     cesna_labels = cesna_labels.tolist()
# #     print(len(cesna_labels))


#     ari_cesna = metrics.adjusted_rand_score(labels_true=labels_true_final, labels_pred=cesna_labels)
#     ami_cesna = metrics.adjusted_mutual_info_score(labels_true=labels_true_final, labels_pred=cesna_labels)
#     nmi_cesna = metrics.normalized_mutual_info_score(labels_true=labels_true_final, labels_pred=cesna_labels)

#     print("results of applying CESNA:", "ari:", '%.2f' % ari_cesna,  "nmi:", '%0.2f' % nmi_cesna)
#     ari_cesna_total.append(ari_cesna)
#     nmi_cesna_total.append(nmi_cesna)

In [ ]:
# print( "%.2f" % np.mean(ari_cesna_total), "%.2f" % np.std(ari_cesna_total), "%.2f" % np.mean(nmi_cesna_total), "%.2f" % np.std(nmi_cesna_total))

0.27(48); 0.25(43); 0.25(0.43); 0.25(43); 0.28(43); 0.29(44); 0.29(44); 0.32(48) (-sb= 0.001)

### Converting Data In GML format (for SIAN and etc)

###### Creating Nodes' attribute dict of dict

In [ ]:
for i in range(10):    for i in range(10):    for i in range(10):    # array2list = []
# for i in range(N):
#     array2list.append(list(Yc_unique[i, 1:]))

# unique_features = []
# for i in array2list:
#     if not i in unique_features:
#         unique_features.append(i)

# # print("unique_features:", unique_features)

# labels_comb_man = ["meta" + str(i+1) for i in range(len(unique_features))]
# labels_comb_man

# Ggml = nx.from_numpy_array(Pin)

# attributes_dict = {}

# label = 1
# labels_true = []
# for uf in unique_features:  #UniqueFeatures
#     tmp_labels = []
#     for i in range(N):
#         if uf == array2list[i]:
#             attributes_dict[i] = {}
#             key = labels_comb_man[label-1]
#             attributes_dict[i][key] = key
#             tmp_labels.append(label)
#     label += 1

# nx.set_node_attributes(Ggml, attributes_dict)
# nx.write_gml(G=Ggml, path="../../data/Lawyers/Lawyers_SIAN.gml")

# with open ("../../data/Lawyers/Lawyers_SIAN.gml", 'r') as fp:
#     GML = fp.readlines()
# len(GML)

# with open ("../../../Newman_2016/Newman_Clauset_code/Lawyers.gml", 'w') as fp:
#     for i in range(len(GML)):
#         if i ==3:
#             fp.write(GML[0])
#             fp.write(GML[1])
#             fp.write(GML[2])
#             fp.write("    ")
#             fp.write(GML[3].split()[0])
#             fp.write(" ")
#             fp.write(GML[4].split()[1])
#             fp.write("\n")
#             fp.write(GML[5])
#         elif "label" in GML[i] and i!=3:
#             fp.write(GML[i-2])
#             fp.write(GML[i-1])
#             fp.write("    ")
#             fp.write(GML[i].split()[0])
#             fp.write(" ")
#             fp.write(GML[i+1].split()[1])
#             fp.write("\n")
#             fp.write(GML[i+2])
#         elif "edge" in GML[i]:
#             fp.write(GML[i])
#             fp.write(GML[i+1])
#             fp.write(GML[i+2])
#             fp.write(GML[i+4])


# print("Conversion is Done!")

In [ ]:
len(unique_features)

### Evaluating the SIAN's results

In [5]:
ari_sian_total, nmi_sian_total = [], []
for i in range(10):
    with open ("/home/Soroosh/Newman_2016/Newman_Clauset_code/lawyers/SIAN_Lawyers_results" + str(i) + ".txt", 'r') as fp:
        SIAN = fp.readlines()

    # labels_true_sorted_final = [item for sublist in labels_true_sorted for item in sublist]
    sian_labels = []
    for line in range(len(SIAN)):
        tmp = SIAN[line].split()
        prob = [float(tmp[t]) for t in range(len(tmp))  if t > 1]
        sian_labels.append(np.argmax(np.asarray(prob)))
    ari_sian = metrics.adjusted_rand_score(labels_true=labels_true_final, labels_pred=sian_labels)
    ami_sian = metrics.adjusted_mutual_info_score(labels_true=labels_true_final, labels_pred=sian_labels)
    nmi_sian = metrics.normalized_mutual_info_score(labels_true=labels_true_final, labels_pred=sian_labels)
    ari_sian_total.append(ari_sian)
    nmi_sian_total.append(nmi_sian)
    print("results of applying SIAN:", "ari:", '%.2f' % ari_sian,  "nmi:", '%0.2f' % nmi_sian)

results of applying SIAN: ari: 0.58 nmi: 0.69
results of applying SIAN: ari: 0.66 nmi: 0.75
results of applying SIAN: ari: 0.60 nmi: 0.70
results of applying SIAN: ari: 0.60 nmi: 0.70
results of applying SIAN: ari: 0.61 nmi: 0.77
results of applying SIAN: ari: 0.67 nmi: 0.76
results of applying SIAN: ari: 0.54 nmi: 0.72
results of applying SIAN: ari: 0.57 nmi: 0.68
results of applying SIAN: ari: 0.57 nmi: 0.68
results of applying SIAN: ari: 0.54 nmi: 0.62


In [6]:
print( "%.2f" % np.mean(ari_sian_total), "%.2f" % np.std(ari_sian_total), "%.2f" % np.mean(nmi_sian_total), "%.2f" % np.std(nmi_sian_total))

0.59 0.04 0.71 0.04
